# Deep Sudoku 

In [ ]:
import numpy as np
import pandas as pd
import keras
import keras.backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
path = '../input/sudoku/sudoku.csv'
data = pd.read_csv(path)
try:
    data = pd.DataFrame({"quizzes":data["puzzle"],"solutions":data["solution"]})
except:
    pass
data.head()

In [ ]:
data.info()

In [ ]:
print("Quiz:\n",np.array(list(map(int,list(data['quizzes'][0])))).reshape(9,9))

In [ ]:
print("Solution:\n",np.array(list(map(int,list(data['solutions'][0])))).reshape(9,9))

In [ ]:
class DataGenerator(Sequence):
    def __init__(self, df, batch_size=16, subset="train"):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.subset = subset
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.floor(len(self.df)/self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        np.random.shuffle(self.indexes)
            
    def __getitem__(self,index):
        
        X = np.empty((self.batch_size, 9,9,1))
        y = np.empty((self.batch_size,81,1))
        
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        for i,f in enumerate(self.df['quizzes'].iloc[indexes]):
            X[i,] = (np.array(list(map(int,list(f)))).reshape((9,9,1))/9)
            
        for i,f in enumerate(self.df['solutions'].iloc[indexes]):
            y[i,] = np.array(list(map(int,list(f)))).reshape((81,1))-1
            
        return X, y

In [ ]:
train_idx = int(len(data)*0.95)
data = data.sample(frac=1).reset_index(drop=True)
training_generator = DataGenerator(data.iloc[:train_idx], batch_size=640)
validation_generator = DataGenerator(data.iloc[train_idx:], batch_size=640)

In [ ]:
model = Sequential([
    Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(9,9,1)),
    Flatten(),
    Dense(81*9*3, activation='relu'),    
    Dropout(.2),
    Dense(81*9*2, activation='relu'),
    Dropout(.2),
    Dense(81*9, activation='relu'),
    Reshape((-1, 9)),
    Activation('softmax'),
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(training_generator, validation_data=validation_generator, epochs=3, verbose=1)

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Épocas')
    plt.ylabel('Erro')
    plt.legend()
    plt.grid(True)


def plot_acc(history):
    plt.plot(history.history['accuracy'], label='acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
plot_acc(history)